# 1.ChatMessgeHistory的使用

In [1]:
from faiss.contrib.ondisk import merge_ondisk
from langchain.memory import ConversationBufferMemory
from langchain_classic.chains.llm import LLMChain
from langchain_community.chat_message_histories import ChatMessageHistory

# 场景1：记忆存储
history = ChatMessageHistory()
# 添加相关的消息进行存储
history.add_user_message("你好")
history.add_ai_message("很高兴认识你")
print(history)

Human: 你好
AI: 很高兴认识你


# 2.对接大模型

In [2]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import FewShotPromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate, \
    MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import os
import dotenv

dotenv.load_dotenv()
# 方式1
os.environ["OPENAI_BASE_URL"]=os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    max_tokens=512,
)
history = ChatMessageHistory()
# 添加相关的消息进行存储
history.add_user_message("你好")
history.add_ai_message("很高兴认识你")
history.add_user_message("帮我计算1+2*3=？")

response = llm.invoke(history.messages)
print(response.content)



根据运算顺序，先进行乘法再进行加法。 

所以，计算过程是：
1 + (2 * 3) = 1 + 6 = 7

因此，1 + 2 * 3 = 7。


# 3.ConversationBufferMemory

In [4]:
# 举例1
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)
memory.save_context(inputs={"input":"你好，我叫小明"},outputs={"output":"很高兴认识你"})
memory.save_context(inputs={"input":"帮我回答一下1+2*3=？"},outputs={"output":"7"})
print(memory.load_memory_variables({}))
# 返回的字典的key叫history
print(memory.chat_memory.messages)


{'history': [HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我回答一下1+2*3=？', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]}


# 举例3:结合大模型

In [13]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# 1. 模型与Prompt不变
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
prompt = PromptTemplate.from_template(
    """
    你可以与人类对话。
    当前对话历史：{history}
    人类问题：{question}
    回复：
    """
)

# 2. 内存 (确保 memory_key 和 prompt 里的 {history} 对应)
memory = ConversationBufferMemory(memory_key="history")

# 3. 【修改点】不要用 |，直接实例化 LLMChain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True # 开启后能在控制台看到历史记录的注入过程
)

# 4. 调用
response = chain.invoke({"question": "你好，我的名字叫小明"})
print(response)
response = chain.invoke({"question": "我叫什么名字"})
print(response)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

    你可以与人类对话。
    当前对话历史：
    人类问题：你好，我的名字叫小明
    回复：
    


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
{'question': '你好，我的名字叫小明', 'history': '', 'text': '你好，小明！很高兴认识你。有什么我可以帮助你的吗？'}
Prompt after formatting:

    你可以与人类对话。
    当前对话历史：Human: 你好，我的名字叫小明
AI: 你好，小明！很高兴认识你。有什么我可以帮助你的吗？
    人类问题：我叫什么名字
    回复：
    

> Finished chain.
{'question': '我叫什么名字', 'history': 'Human: 你好，我的名字叫小明\nAI: 你好，小明！很高兴认识你。有什么我可以帮助你的吗？', 'text': '你叫小明。'}


# 举例4 ChatPromptTemplate

In [18]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import FewShotPromptTemplate, ChatPromptTemplate,MessagesPlaceholder
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# 1. 模型与Prompt不变
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
prompt = ChatPromptTemplate.from_messages([
    ("system","你是一个与人类对话的机器人。"),
    MessagesPlaceholder(variable_name="history"),
    ("human","问题：{question}"),

])

# 2. 内存 (确保 memory_key 和 prompt 里的 {history} 对应)
memory = ConversationBufferMemory(memory_key="history",return_messages=True)

# 3. 【修改点】不要用 |，直接实例化 LLMChain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True # 开启后能在控制台看到历史记录的注入过程
)

# 4. 调用
response = chain.invoke({"question": "中国首度在哪"})
print(response)
response = chain.invoke({"question": "我刚才问了什么问题"})
print(response)


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
System: 你是一个与人类对话的机器人。
Human: 问题：中国首度在哪


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
{'question': '中国首度在哪', 'history': [HumanMessage(content='中国首度在哪', additional_kwargs={}, response_metadata={}), AIMessage(content='“中国首度”通常是指中国首次在某个领域或事件上取得的成就，例如首次发射载人航天飞船、首次成功举办奥运会等。如果你能提供更具体的上下文或领域，我将更好地回答你的问题。', additional_kwargs={}, response_metadata={})], 'text': '“中国首度”通常是指中国首次在某个领域或事件上取得的成就，例如首次发射载人航天飞船、首次成功举办奥运会等。如果你能提供更具体的上下文或领域，我将更好地回答你的问题。'}
Prompt after formatting:
System: 你是一个与人类对话的机器人。
Human: 中国首度在哪
AI: “中国首度”通常是指中国首次在某个领域或事件上取得的成就，例如首次发射载人航天飞船、首次成功举办奥运会等。如果你能提供更具体的上下文或领域，我将更好地回答你的问题。
Human: 问题：我刚才问了什么问题

> Finished chain.
{'question': '我刚才问了什么问题', 'history': [HumanMessage(content='中国首度在哪', additional_kwargs={}, response_metadata={}), AIMessage(content='“中国首度”通常是指中国首次在某个领域或事件上取得的成就，例如首次发射载人航天飞船、首次成功举办奥运会等。如果你能提供更具体的上下文或领域，我将更好地回答你的问题。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚才问了什么问题', additional_kwargs={}, response_metadata={}), AIMessage(content='你问的是“中国首度在哪”。这个问题比较模糊，可能是在询问中国首次在某个领域或事件上的成就。如果你有更具体的内容或想法，请告诉我，我会更好地帮助你！', a

# ConversationChain

In [21]:
from langchain.chains.conversation.base import ConversationChain
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import FewShotPromptTemplate, ChatPromptTemplate,MessagesPlaceholder
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# 1. 模型与Prompt不变
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
prompt = PromptTemplate.from_template(
    """
    你可以与人类对话。
    当前对话历史：{history}
    人类问题：{input}
    回复：
    """
)

# # 2. 内存 (确保 memory_key 和 prompt 里的 {history} 对应)
# memory = ConversationBufferMemory(memory_key="history",return_messages=True)
#
# # 3. 【修改点】不要用 |，直接实例化 LLMChain
# chain = LLMChain(
#     llm=llm,
#     prompt=prompt,
#     memory=memory,
#     verbose=True # 开启后能在控制台看到历史记录的注入过程
# )
chain = ConversationChain(llm=llm,prompt=prompt)
# 4. 调用
response = chain.invoke({"input": "中国首度在哪"})
print(response)
response = chain.invoke({"input": "我刚才问了什么问题"})
print(response)


{'input': '中国首度在哪', 'history': '', 'response': '中国首度是指中国首次的某项活动或事件，具体要看你指的是什么。如果你是在问中国的首次历史事件，比如中国的首次建国、首次登月等，可以提供更具体的信息，以便我更好地回答你！'}
{'input': '我刚才问了什么问题', 'history': 'Human: 中国首度在哪\nAI: 中国首度是指中国首次的某项活动或事件，具体要看你指的是什么。如果你是在问中国的首次历史事件，比如中国的首次建国、首次登月等，可以提供更具体的信息，以便我更好地回答你！', 'response': '你刚才问的问题是“中国首度在哪”。你想了解的是中国首次的某项活动或事件。如果有具体的内容或背景，欢迎告诉我！'}


# 使用默认的提示词模板

In [22]:
from langchain.chains.conversation.base import ConversationChain
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import FewShotPromptTemplate, ChatPromptTemplate,MessagesPlaceholder
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# 1. 模型与Prompt不变
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
# prompt = PromptTemplate.from_template(
#     """
#     你可以与人类对话。
#     当前对话历史：{history}
#     人类问题：{input}
#     回复：
#     """
# )

# # 2. 内存 (确保 memory_key 和 prompt 里的 {history} 对应)
# memory = ConversationBufferMemory(memory_key="history",return_messages=True)
#
# # 3. 【修改点】不要用 |，直接实例化 LLMChain
# chain = LLMChain(
#     llm=llm,
#     prompt=prompt,
#     memory=memory,
#     verbose=True # 开启后能在控制台看到历史记录的注入过程
# )
chain = ConversationChain(llm=llm)
# 4. 调用
response = chain.invoke({"input": "中国首度在哪"})
print(response)
response = chain.invoke({"input": "我刚才问了什么问题"})
print(response)


{'input': '中国首度在哪', 'history': '', 'response': '中国首度是指中国首次进行某项重要活动或事件。具体来说，"首度"这个词可以用在许多上下文中，比如首度参与国际比赛、首度发射卫星、首度举办大型活动等等。如果你能提供更具体的上下文或事件，我会更乐意帮助你找到相关的信息！'}
{'input': '我刚才问了什么问题', 'history': 'Human: 中国首度在哪\nAI: 中国首度是指中国首次进行某项重要活动或事件。具体来说，"首度"这个词可以用在许多上下文中，比如首度参与国际比赛、首度发射卫星、首度举办大型活动等等。如果你能提供更具体的上下文或事件，我会更乐意帮助你找到相关的信息！', 'response': '你刚才问了“中国首度在哪”。这个问题有点模糊，不太清楚你具体是指哪方面的“首度”。如果你能提供更多的背景信息或者具体的事件，我会更好地回答你的问题！'}


# ConversationBufferWindowMemory

In [30]:
from langchain.memory import ConversationBufferWindowMemory
# 1.导入相关包
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
# 2.定义模版
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。
当前对话：
{history}
Human: {question}
AI:"""
# 3.定义提示词模版
prompt_template = PromptTemplate.from_template(template)
# 4.创建大模型
llm = ChatOpenAI(model="gpt-4o-mini")
# 5.实例化ConversationBufferWindowMemory对象，设定窗口阈值
memory = ConversationBufferWindowMemory(k=1)
# 6.定义LLMChain
conversation_with_summary = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    # verbose=True,

)
# 7.执行链（第一次提问）
respon1 = conversation_with_summary.invoke({"question":"你好，我是孙小空"})
# print(respon1)
# 8.执行链（第二次提问）
respon2 =conversation_with_summary.invoke({"question":"我还有两个师弟，一个是猪小戒，一个是沙小僧"})
# print(respon2)
# 9.执行链（第三次提问）
respon3 =conversation_with_summary.invoke({"question":"我今年高考，竟然考上了1本"})
# print(respon3)
# 10.执行链（第四次提问）
respon4 =conversation_with_summary.invoke({"question":"我叫什么？"})
print(respon4)

{'question': '我叫什么？', 'history': 'Human: 我今年高考，竟然考上了1本\nAI: 恭喜你！考上一本大学是一个了不起的成就，值得庆祝！你选择了哪个专业呢？或者你对未来的学习和生活有什么期待吗？', 'text': '抱歉，我不知道你的名字。你愿意告诉我吗？或者我们可以聊聊其他话题！'}
